# Azure AI Foundry Agent Service



https://learn.microsoft.com/en-us/azure/ai-foundry/agents/overview

https://learn.microsoft.com/en-us/python/api/overview/azure/ai-agents-readme?view=azure-python

## Data Agent Connector

https://learn.microsoft.com/en-us/fabric/data-science/how-to-consume-data-agent

https://learn.microsoft.com/en-us/azure/ai-foundry/agents/how-totools/fabric?pivots=rest

In [4]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import FabricTool, ListSortOrder
from dotenv import load_dotenv  

load_dotenv(override=True) 

True

In [5]:
system_message = """You are a helpful agent that can query data from a SQL database using the Fabric tool."""
conn_id = os.environ["AZURE_FABRIC_CONNECTION_ID"]

# Initialize agent Fabric tool and add the connection ID
fabric = FabricTool(connection_id=conn_id)

# Retrieve the project endpoint from environment variables
project_endpoint = os.environ["PROJECT_ENDPOINT"]

# Initialize the AIProjectClient
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),
)


In [6]:
agent_name = os.environ["AZURE_AI_FABRIC_AGENT_NAME"]
agent_list = project_client.agents.list_agents()

agent = None
if agent_list:
    for agent_object in agent_list:
        if agent_object.name == agent_name:
            agent = agent_object

if not agent:
    print(f"Agent '{agent_name}' not found in the project.")
    agent = project_client.agents.create_agent(
        model=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
        name=agent_name,
        tools=fabric.definitions,
        instructions=system_message,
        headers={"x-ms-enable-preview": "true"},
    )
    print(f"Created agent, ID: {agent.id}")
else:
    print(f"Agent '{agent_name}' already exists, ID: {agent.id}")

Agent 'AIFoundryDataAgent' already exists, ID: asst_zj5mEkm9wYHsiHyfpYWovIrz


In [7]:
# Create a thread for communication
thread = project_client.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

# Send a message to the thread
message = project_client.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content="What are the tables in the warehoues?",
)
print(f"Created message, ID: {message['id']}")

Created thread, ID: thread_BqA4HRExmKuHNFPHM2svo73k
Created message, ID: msg_k1sK1Lh2a8UEQew2lgxYmjNl


In [8]:
run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

print("agent name = " + agent.name)
print("agent id: " + agent.id)
print("run id" + run.id)

Run finished with status: RunStatus.COMPLETED
agent name = AIFoundryDataAgent
agent id: asst_zj5mEkm9wYHsiHyfpYWovIrz
run idrun_q0fHc46eGs48eGGXegHL8wXX


In [ ]:
if run.status == "failed":
        print(f"Run failed: {run.last_error}")
    
# Fetch and log all messages
messages = project_client.agents.messages.list(thread_id=thread.id)
for message in messages:
    print(f"Role: {message.role}, Content: {message.content}")

run_steps = project_client.agents.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
        # print(f"Step {step['id']} status: {step['status']}")
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  Tool calls:")
            for call in tool_calls:
                print(call)
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

              

In [ ]:
message = project_client.agents.messages.create(
        thread_id=thread.id,
        role="user",  # Role of the message sender
        content="what are 10 common zip codes",  # Message content
    )

In [ ]:
run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

print(agent.name)
print(agent.id)

In [ ]:
if run.status == "failed":
        print(f"Run failed: {run.last_error}")
    
# Fetch and log all messages
messages = project_client.agents.messages.list(thread_id=thread.id)
for message in messages:
    print(f"Role: {message.role}, Content: {message.content}")

run_steps = project_client.agents.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
        # print(f"Step {step['id']} status: {step['status']}")
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  Tool calls:")
            for call in tool_calls:
                print(call)
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

              